<a href="https://colab.research.google.com/github/3Techies/Bias-Detection-ML-models/blob/main/notebooks/02_baseline_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/3Techies/Bias-Detection-ML-models.git


fatal: destination path 'Bias-Detection-ML-models' already exists and is not an empty directory.


In [2]:
%cd Bias-Detection-ML-models/notebooks

/content/Bias-Detection-ML-models/notebooks


In [3]:
import sys
import os

print("CWD:", os.getcwd())

sys.path.append(os.path.abspath(".."))

print("Files in root:", os.listdir(".."))

import config
print("SUCCESS: config imported")

CWD: /content/Bias-Detection-ML-models/notebooks
Files in root: ['__pycache__', 'data', '.git', 'README.md', 'LICENSE', 'notebooks', 'config.py', 'results', 'how-to-use-git.md', '.gitignore']
SUCCESS: config imported


In [4]:
import pandas as pd
%cd ..
print(config.DATA_PATH)

/content/Bias-Detection-ML-models
data/loan_data.csv


In [5]:
df = pd.read_csv(config.DATA_PATH)
df.head(10)

,person_age,person_gender,person_education,person_income,person_emp_exp,person_home_ownership,loan_amnt,loan_intent,loan_int_rate,loan_percent_income,cb_person_cred_hist_length,credit_score,previous_loan_defaults_on_file,loan_status
0,22.0,female,Master,71948.0,0,RENT,35000.0,PERSONAL,16.02,0.49,3.0,561,No,1
1,21.0,female,High School,12282.0,0,OWN,1000.0,EDUCATION,11.14,0.08,2.0,504,Yes,0
2,25.0,female,High School,12438.0,3,MORTGAGE,5500.0,MEDICAL,12.87,0.44,3.0,635,No,1
3,23.0,female,Bachelor,79753.0,0,RENT,35000.0,MEDICAL,15.23,0.44,2.0,675,No,1
4,24.0,male,Master,66135.0,1,RENT,35000.0,MEDICAL,14.27,0.53,4.0,586,No,1
5,21.0,female,High School,12951.0,0,OWN,2500.0,VENTURE,7.14,0.19,2.0,532,No,1
6,26.0,female,Bachelor,93471.0,1,RENT,35000.0,EDUCATION,12.42,0.37,3.0,701,No,1
7,24.0,female,High School,95550.0,5,RENT,35000.0,MEDICAL,11.11,0.37,4.0,585,No,1
8,24.0,female,Associate,100684.0,3,RENT,35000.0,PERSONAL,8.90,0.35,2.0,544,No,1
9,21.0,female,High School,12739.0,0,OWN,1600.0,VENTURE,14.74,0.13,3.0,640,No,1


In [6]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [7]:
target_column="loan_status"  #selecting the target_column.
X = df.drop(columns=[target_column]) #isolate the features from dataset,by droping the target feature.
y = df[target_column] #giving y the df of target column

In [8]:
categorical_cols = X.select_dtypes(include=["object"]).columns.tolist()#The categorical columns from the input features.
numerical_cols = X.select_dtypes(include=["int64", "float64"]).columns.tolist() #The numerical columns from the input features.

print("categorical columns: ",categorical_cols)
print("numerical columns: ",numerical_cols)


categorical columns:  ['person_gender', 'person_education', 'person_home_ownership', 'loan_intent', 'previous_loan_defaults_on_file']
numerical columns:  ['person_age', 'person_income', 'person_emp_exp', 'loan_amnt', 'loan_int_rate', 'loan_percent_income', 'cb_person_cred_hist_length', 'credit_score']


In [9]:
preprocessor = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_cols),
        ("num", "passthrough", numerical_cols)
    ]
)


In [10]:
X_train, X_test, y_train, y_test = train_test_split( #defining the training dataset and the testing dataset.
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)


In [11]:
dt_model = DecisionTreeClassifier(  #describing the model with our requirements.
    criterion="gini",
    max_depth=6,
    min_samples_leaf=50,
    random_state=42
)

In [12]:
rf_model = RandomForestClassifier(
    n_estimators=200,
    criterion="gini",
    max_depth=6)
xgb_model = XGBClassifier(
    n_estimators=200,
    learning_rate=0.1,
    max_depth=5,
    subsample=0.8,
    colsample_bytree=0.8,
    objective='binary:logistic',
    random_state=42)
gr_model = GradientBoostingClassifier(
    n_estimators=200,
    learning_rate=0.1,
    max_depth=6
)

In [13]:
pipeline=Pipeline(
    steps=[
        ("preprocessing", preprocessor),
        ("model", dt_model)
    ]
)

In [14]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['person_gender',
                                                   'person_education',
                                                   'person_home_ownership',
                                                   'loan_intent',
                                                   'previous_loan_defaults_on_file']),
                                                 ('num', 'passthrough',
                                                  ['person_age',
                                                   'person_income',
                                                   'person_emp_exp',
                                                   'loan_amnt', 'loan_int_rate',
                                                   'loan_percent_income',
                                                   'cb_person_cred_hist_length',
                                                   'credit_score'])])),
                ('model',
                 DecisionTreeClassifier(max_depth=6, min_samples_leaf=50,
                                        random_state=42))])

In [15]:
y_pred_dt = pipeline.predict(X_test) #Finally testing the dT model by providing it with testing data.
print(y_pred_dt[:5])

[0 0 0 0 1]


In [16]:
print("Accuracy:", accuracy_score(y_test, y_pred_dt)*100,"%") #calculating the accuracy of the model.
print("\nClassification Report:\n", classification_report(y_test, y_pred_dt))

Accuracy: 91.2 %

Classification Report:
               precision    recall  f1-score   support

           0       0.92      0.97      0.94      7000
           1       0.86      0.72      0.78      2000

    accuracy                           0.91      9000
   macro avg       0.89      0.84      0.86      9000
weighted avg       0.91      0.91      0.91      9000



In [17]:
pipeline=Pipeline(
    steps=[
        ("preprocessing", preprocessor),
        ("model", rf_model)
    ]
)
pipeline.fit(X_train,y_train)

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['person_gender',
                                                   'person_education',
                                                   'person_home_ownership',
                                                   'loan_intent',
                                                   'previous_loan_defaults_on_file']),
                                                 ('num', 'passthrough',
                                                  ['person_age',
                                                   'person_income',
                                                   'person_emp_exp',
                                                   'loan_amnt', 'loan_int_rate',
                                                   'loan_percent_income',
                                                   'cb_person_cred_hist_length',
                                                   'credit_score'])])),
                ('model',
                 RandomForestClassifier(max_depth=6, n_estimators=200))])

In [18]:
y_pred_rf = pipeline.predict(X_test) #Finally testing the dT model by providing it with testing data.
print(y_pred_rf[:5])
print("Accuracy:", accuracy_score(y_test, y_pred_rf)*100,"%") #calculating the accuracy of the model.
print("\nClassification Report:\n", classification_report(y_test, y_pred_rf))

[0 0 0 0 1]
Accuracy: 91.9 %

Classification Report:
               precision    recall  f1-score   support

           0       0.92      0.98      0.95      7000
           1       0.91      0.70      0.79      2000

    accuracy                           0.92      9000
   macro avg       0.92      0.84      0.87      9000
weighted avg       0.92      0.92      0.92      9000



In [19]:
pipeline=Pipeline(
    steps=[
        ("preprocessing", preprocessor),
        ("model", xgb_model)
    ]
)
pipeline.fit(X_train,y_train)

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['person_gender',
                                                   'person_education',
                                                   'person_home_ownership',
                                                   'loan_intent',
                                                   'previous_loan_defaults_on_file']),
                                                 ('num', 'passthrough',
                                                  ['person_age',
                                                   'person_income',
                                                   'person_emp_exp',
                                                   'loan_amnt', 'loan_int_rate',
                                                   'loan_percent_income',
                                                   'cb_person_c...
                               feature_types=None, feature_weights=None,
                               gamma=None, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None, learning_rate=0.1,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=5, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, multi_strategy=None,
                               n_estimators=200, n_jobs=None,
                               num_parallel_tree=None, ...))])

In [20]:
y_pred_xgb = pipeline.predict(X_test) #Finally testing the dT model by providing it with testing data.
print(y_pred_xgb[:5])
print("Accuracy:", accuracy_score(y_test, y_pred_xgb)*100,"%") #calculating the accuracy of the model.
print("\nClassification Report:\n", classification_report(y_test, y_pred_xgb))

[0 0 0 0 1]
Accuracy: 93.4 %

Classification Report:
               precision    recall  f1-score   support

           0       0.94      0.97      0.96      7000
           1       0.90      0.80      0.84      2000

    accuracy                           0.93      9000
   macro avg       0.92      0.88      0.90      9000
weighted avg       0.93      0.93      0.93      9000



In [21]:
pipeline=Pipeline(
    steps=[
        ("preprocessing", preprocessor),
        ("model", gr_model)
    ]
)
pipeline.fit(X_train,y_train)

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['person_gender',
                                                   'person_education',
                                                   'person_home_ownership',
                                                   'loan_intent',
                                                   'previous_loan_defaults_on_file']),
                                                 ('num', 'passthrough',
                                                  ['person_age',
                                                   'person_income',
                                                   'person_emp_exp',
                                                   'loan_amnt', 'loan_int_rate',
                                                   'loan_percent_income',
                                                   'cb_person_cred_hist_length',
                                                   'credit_score'])])),
                ('model',
                 GradientBoostingClassifier(max_depth=6, n_estimators=200))])

In [22]:
y_pred_gr = pipeline.predict(X_test) #Finally testing the dT model by providing it with testing data.
print(y_pred_gr[:5])
print("Accuracy:", accuracy_score(y_test, y_pred_gr)*100,"%") #calculating the accuracy of the model.
print("\nClassification Report:\n", classification_report(y_test, y_pred_gr))

[0 0 0 0 1]
Accuracy: 93.72222222222221 %

Classification Report:
               precision    recall  f1-score   support

           0       0.94      0.98      0.96      7000
           1       0.91      0.80      0.85      2000

    accuracy                           0.94      9000
   macro avg       0.93      0.89      0.91      9000
weighted avg       0.94      0.94      0.94      9000

